In [1]:
from PIL import Image
import os
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential, Model
from keras.layers import Dense, Embedding, Reshape, Conv2DTranspose
from keras.layers import Input, Conv2D, Flatten, Dense, Embedding, Concatenate
from tensorflow.keras.layers import Dense, Reshape, Flatten, Conv2D, Conv2DTranspose
from keras.preprocessing import image
from keras.applications.densenet import preprocess_input

In [2]:

# Diretório que contém as imagens
diretorio = "mapas/padronizadas/"
txt_file = "mapas/padronizadas/etiquetas.txt"

def load_images_and_labels(diretorio, txt_file):
    image_paths = []
    labels = []

    with open(txt_file, 'r') as file:
        lines = file.readlines()
        for line in lines:
            parts = line.strip().split()
            if len(parts) >= 2:  # Verificar se há pelo menos dois elementos na linha
                image_path = os.path.join(diretorio, parts[0])  # O primeiro elemento é o nome da imagem
                label = parts[1]  # O segundo elemento é a etiqueta
                image_paths.append(image_path)
                labels.append(label)

    return image_paths, labels

image_paths, labels = load_images_and_labels(diretorio, txt_file)

# Pré-processamento de imagens (redimensionamento e normalização)
def preprocess_image(image_path, target_size):
    img = image.load_img(image_path, target_size=target_size)
    img = image.img_to_array(img)
    img = preprocess_input(img)
    return img

target_size = (256, 256)  # Tamanho desejado das imagens
dados_preprocessados = [preprocess_image(path, target_size) for path in image_paths]
dados_preprocessados = np.array(dados_preprocessados)
dados_preprocessados[5]

array([[[ 1.8721637 ,  1.4656863 ,  1.3502399 ],
        [ 1.4611696 ,  1.0280112 ,  0.7576472 ],
        [ 1.8379141 ,  1.4306723 ,  0.8970808 ],
        ...,
        [ 2.0776608 ,  2.1134453 ,  1.6813945 ],
        [ 2.2489083 ,  2.3410363 ,  1.8905448 ],
        [ 2.11191   ,  2.2184873 ,  1.7685405 ]],

       [[ 1.6666666 ,  1.2731093 ,  1.1236603 ],
        [ 0.8960528 ,  0.45028022,  0.18248373],
        [ 1.5296686 ,  1.0980393 ,  0.5310677 ],
        ...,
        [ 1.9064132 ,  1.9033613 ,  1.4896734 ],
        [ 1.5981677 ,  1.6232493 ,  1.193377  ],
        [ 1.3412963 ,  1.4131653 ,  0.9667976 ]],

       [[ 1.5639181 ,  1.1155462 ,  0.94936836],
        [ 1.2727973 ,  0.81792724,  0.51363856],
        [ 1.6152923 ,  1.1855743 ,  0.6007845 ],
        ...,
        [ 1.3412963 ,  1.2731093 ,  0.8796516 ],
        [ 1.3755459 ,  1.3606442 ,  0.94936836],
        [ 1.6152923 ,  1.6407562 ,  1.1759479 ]],

       ...,

       [[ 1.8207895 ,  1.9733893 ,  1.8208281 ],
        [ 0

In [3]:
input_dim = 100  # Mantenha a dimensão do vetor de entrada
target_size = (256, 256)  # Tamanho desejado das imagens

def build_generator(input_dim, target_size):
    model = Sequential()
    model.add(Dense(128, input_dim=input_dim))
    model.add(Dense(np.prod(target_size) * 3, activation='tanh'))  # Saída compatível com 256x256x3
    model.add(Reshape(target_size + (3,)))  # Redimensiona para 256x256x3

    return model

generator = build_generator(input_dim, target_size)

In [4]:
def build_discriminator(input_shape):
    model = Sequential()
    model.add(Conv2D(64, (4, 4), strides=(2, 2), padding='same', input_shape=input_shape, activation='relu'))
    model.add(Conv2D(128, (4, 4), strides=(2, 2), padding='same', activation='relu'))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

    return model

# Tamanho da imagem de entrada (deve corresponder ao tamanho das imagens que você está gerando)
input_shape = (256, 256, 3)
discriminator = build_discriminator(input_shape)
discriminator.compile(loss='binary_crossentropy', optimizer='adam')

In [5]:
# Congelar o discriminador durante o treinamento da GAN
discriminator.trainable = False

# Entrada para a GAN
gan_input = Input(shape=(input_dim,))
generated_image = generator(gan_input)

# Saída da GAN (a saída do gerador passa pelo discriminador)
gan_output = discriminator(generated_image)

# Crie o modelo da GAN
gan = Model(gan_input, gan_output)

# Compile a GAN
gan.compile(loss='binary_crossentropy', optimizer='adam')

In [6]:
def save_generated_images(epoch, generator, examples=10, dim=(1, 10), figsize=(10, 1)):
    noise = np.random.normal(0, 1, (examples, input_dim))
    generated_images = generator.predict(noise)
    
    # Normaliza as imagens geradas para o intervalo [0, 1]
    generated_images = 0.5 * generated_images + 0.5
    
    plt.figure(figsize=figsize)
    for i in range(examples):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generated_images[i], interpolation='nearest')
        plt.axis('off')
    
    plt.tight_layout()
    plt.savefig(f'generated_image_epoch_{epoch}.png')
    plt.close()

In [13]:
dados_preprocessados = np.array(dados_preprocessados)

d_losses = []
g_losses = []

# Parâmetros de treinamento
epochs = 100  # Número de épocas
batch_size = 32  # Tamanho do lote
sample_interval = 100  # Intervalo para salvar imagens de exemplo

for epoch in range(epochs):
    # Treinar o discriminador
    idx = np.random.randint(0, len(dados_preprocessados), batch_size)
    real_images = dados_preprocessados[idx]
    noise = np.random.normal(0, 1, (batch_size, input_dim))
    generated_images = generator.predict(noise)

    real_labels = np.ones((batch_size, 1))
    fake_labels = np.zeros((batch_size, 1))

    d_loss_real = discriminator.train_on_batch(real_images, real_labels)
    d_loss_fake = discriminator.train_on_batch(generated_images, fake_labels)

    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # Treinar a GAN
    noise = np.random.normal(0, 1, (batch_size, input_dim))
    valid_labels = np.ones((batch_size, 1))

    g_loss = gan.train_on_batch(noise, valid_labels)

    # Imprimir progresso a cada sample_interval épocas
    if epoch % (sample_interval * 10) == 0:
        d_losses.append(d_loss)
        g_losses.append(g_loss)
        save_generated_images(epoch, generator, examples=10, dim=(1, 10), figsize=(10, 1))
        
d_losses

1/1 [==============================] - 0s 51ms/step


[2.156014488718938e-05]

In [14]:
g_losses

[10.662208557128906]

In [18]:
# Tamanho do vetor de ruído (input)
input_dim = 100

# Crie um vetor de ruído aleatório
noise = np.random.normal(0, 1, (1, input_dim))

# Gere uma imagem usando o gerador
generated_image = generator.predict(noise)

generated_image = generated_image[0]

# Converta a imagem de float (entre 0 e 1) para int (entre 0 e 255)
generated_image = (generated_image * 255).astype(np.uint8)

# Crie uma imagem PIL a partir da matriz NumPy
image = Image.fromarray(generated_image)

# Salve a imagem com o nome "ximg.png" (ou outro formato de imagem, como .jpg, .png, etc.)
image.save("ximg.png")

1/1 [==============================] - 0s 99ms/step


In [17]:
nome_do_arquivo = 'ganMap.keras'

# Salve o modelo GAN completo (incluindo gerador e discriminador) em um único arquivo
gan.save(nome_do_arquivo)


In [ ]:
modelo_gan_carregado = load_model(ganMap.keras)